In [ ]:
#RESUME SUMMARIZATION USING NLP
#BSCS 3AB | GROUP  12 | CS353
#COMPRA,IMEE
#DALUMPINES, SOPHIA
#DELOS REYES, CHRISTINE
#FLOR, LEA


# install spacy_transformers and update spacy library
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Import necessary packages
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
!nvidia-smi

Tue Jan 31 05:41:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    30W /  70W |    334MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# load training data from a JSON file
data = json.load(open('/content/ResumeSummarization/data/training/train_data.json', 'r'))

In [ ]:
# get the number of training data
len(data)

200

In [ ]:
# initialize a spacy model
!python -m spacy init fill-config /content/ResumeSummarization/data/training/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# show the n data in the training set
data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [ ]:
# define the function to get spaCy documents with entities
def get_spacy_doc(file,input_data):
    nlp = spacy.blank('en')
    doc_bin = DocBin()
    
    # loop through the data to create spaCy documents with entities
    for text, annotations in tqdm(input_data):
        docu = nlp(text)
        annotations = annotations['entities']
        
        entities = []
        indices= []
        
        # loop through the entities to add them to the spaCy document
        for start, end, label in annotations:
            skip = False
            for index in range(start, end):
                if index in indices:
                    skip = True
                    break

            # Skip the current entity if it overlaps with a previous entity        
            if skip == True:
                continue

            # Add the indices of the current entity to the list    
            indices = indices + list(range(start, end))
            
            try:
                # Create a spaCy Span object from the start and end indices
                span = docu.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue
                
            if span is None:
                # Write the error information to the file
                err_text = str([start, end]) + "   " + str(text) + "\n"
                file.write(err_text)
                
            else:
                entities.append(span)
                
        try:
            docu.ents = entities
            doc_bin.add(docu)
        except:
            pass
        
    return doc_bin

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size = 0.2)

In [ ]:
# Print the number of training and testing examples
len(train_data), len(test_data)

(160, 40)

In [ ]:
# Open a file to write error information
file = open('err.txt', 'w', encoding='utf-8')

# Get the spaCy DocBin for the training data and save it to disk
train_bin = get_spacy_doc(file, train_data)
train_bin.to_disk('train_data.spacy')

# Get the spaCy DocBin for the testing data and save it to disk
test_bin = get_spacy_doc(file, test_data)
test_bin.to_disk('test_data.spacy')

# Close the error file
file.close()

100%|██████████| 40/40 [00:00<00:00, 93.03it/s]


In [ ]:
# Print the number of tokens in the testing DocBin
len(test_bin.tokens)

40

In [ ]:
#Train the model with the training data and config file
!python -m spacy train config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-01-31 05:45:04,466] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2023-01-31 05:45:04,476] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2023-01-31 05:45:04,479] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2023-01-31 05:45:04,480] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassifi

In [ ]:
#Load the best model from the training output
nlp_model = spacy.load('/content/output/model-best')

In [ ]:
#Apply the loaded model to the test sentence
doc = nlp_model('My name is Phoebe Buffay. I worked at Microsoft for 15 years. I have expertise in Web Development and Software Engineering.')
for ent in doc.ents:
  print(ent.label_.replace(" ", " ") + " : ", ent.text)

Name :  Phoebe Buffay
Companies worked at :  Microsoft


In [ ]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys, fitz

In [ ]:
file_name = '/content/ResumeSummarization/data/test/Flor-CV.pdf'
pdf = fitz.open(file_name)

In [ ]:
text = " "
#Loop through the pages in the pdf and add their content to the content variable
for page in pdf:
  text = text + str(page.get_text())

In [ ]:
#Replace multiple spaces with a single space
text = ' '.join(text.split())

In [ ]:
text

'Lea Flor Programmer Manila, Philippines ● 4 years of experience in programming ● Waitressing: Answer customer inquiries and resolve issues, and consistently adhered to quality expectations and standards WORK EXPERIENCE Junior Engineer Teleperformance – Manila May 2015 to September 2019 EDUCATION Technological University of the Philippines SKILLS Programmming'

In [ ]:
#Apply the loaded NLP model to the content of the pdf file
doc = nlp_model(text)
for ent in doc.ents:
  print(ent.label_.replace(" ", " ") + " : ", ent.text)

Name :  Lea Flor
Location :  Manila
Years of Experience :  4 years
Designation :  Junior Engineer
Location :  Manila
College Name :  Technological University of the Philippines
